# Advent of Code 2025
## Python Solutions

### Day 01: Secret Entrance

In [1]:
# part 1
from pathlib import Path

lines = Path("d01.txt").read_text().splitlines()

ptr = 50
N = 100
counter = 0

def rotate(direction: str, number: int, ptr: int, counter: int) -> tuple[int, int]:
    if direction == "L":
        ptr =  (ptr - number) % N
    else: # direction == "R" 
        ptr =  (ptr + number) % N

    if ptr == 0:
        counter += 1

    return ptr, counter

for move in lines:
    direction = move[:1]
    number = int(move[1:])

    ptr, counter = rotate(direction, number, ptr, counter)


counter

992

In [2]:
# part 2
lines = Path("d01.txt").read_text().splitlines()

ptr = 50
N = 100
counter = 0

def rotate(direction: str, number: int, ptr: int, counter: int) -> tuple[int, int]:
    if direction ==  "L":
        for _ in range(number):
            ptr = (ptr - 1) % N
            if ptr == 0:
                counter += 1
    else: # direction == "R"
        for _ in range(number):
            ptr = (ptr + 1) % N
            if ptr == 0:
                counter += 1

    return ptr, counter

for move in lines:
    direction = move[:1]
    number = int(move[1:])

    ptr, counter = rotate(direction, number, ptr, counter)

counter

6133

### Day 02: Gift Shop

In [3]:
# part 1
from pathlib import Path
content = Path("d02.txt").read_text()


# repeated twice:
# 55 (5 twice), 6464 (64 twice), 123123 (123 twice) are invalid IDs
def is_invalid_id(num: int) -> bool:
    s_num = str(num)
    if len(s_num) % 2 == 0:  # only even-length numbers can be repeated twice
        mid = len(s_num) // 2
        return s_num[:mid] == s_num[mid:]
    return False

def sum_invalid_ids(payload: str) -> int:
    ranges = payload.strip().split(",")
    total = 0
    for r in ranges:
        start, end = map(int, r.split("-"))
        for num in range(start, end + 1):
            if is_invalid_id(num):
                total += num
    return total

n_invalid = sum_invalid_ids(content)
n_invalid

30608905813

In [4]:
# part 2
content = Path("d02.txt").read_text()

# repeated at least twice: 
# 12341234 (1234 two times), 123123123 (123 three times), 1212121212 (12 five times), and 1111111 (1 seven times) are invalid IDs
def is_invalid_id(num: int) -> bool:
    s_num = str(num)
    length = len(s_num)
    # check all possible substring lengths
    for k in range(1, length // 2 + 1):
        if length % k == 0:
            if s_num == s_num[:k] * (length // k):
                return True
    return False

def sum_invalid_ids(payload: str) -> int:
    ranges = payload.strip().split(",")
    total = 0
    for r in ranges:
        start, end = map(int, r.split("-"))
        for num in range(start, end + 1):
            if is_invalid_id(num):
                total += num
    return total

n_invalid = sum_invalid_ids(content)
n_invalid

31898925685

**Alternative Solution: Regex**

Part 1: repeated twice
* Regex: `^(\d+)\1$`
* `(\d+)` captures one or more digits
* `\1` matches the same sequence again
* `^`...`$` anchors match the whole string

Part 2: at least twice
* Regex: `^(\d+)\1+$`
* same as above
* `\1+` captured group repeats one or more times

In [5]:
from typing import Callable
import re

pattern_part1 = re.compile(r'^(\d+)\1$')      # exactly two repeats
pattern_part2 = re.compile(r'^(\d+)\1+$')     # at least two repeats

def is_invalid1(num: int) -> bool:
    return bool(pattern_part1.match(str(num)))

def is_invalid2(num: int) -> bool:
    return bool(pattern_part2.match(str(num)))

def sum_invalid_ids(payload: str, valid_func: Callable[[int], bool]) -> int:
    ranges = payload.strip().split(',')
    total = 0
    for r in ranges:
        start, end = map(int, r.split('-'))
        for num in range(start, end + 1):
            if valid_func(num):
                total += num
    return total

content = Path("d02.txt").read_text()
n_invalid_1 = sum_invalid_ids(content, is_invalid1)
n_invalid_2 = sum_invalid_ids(content, is_invalid2)

print("Part 1: ", n_invalid_1)
print("Part 2: ", n_invalid_2)

Part 1:  30608905813
Part 2:  31898925685


### Day 03: Lobby

In [6]:
# part 1
from pathlib import Path

def sum_max_joltage(content: str) -> int:
    total = 0
    for line in content.splitlines():
        digits = list(map(int, line.strip()))
        # two largest digits in order
        max_jolt = 0
        for i in range(len(digits)):
            for j in range(i + 1, len(digits)):
                jolt = digits[i] * 10 + digits[j]
                if jolt > max_jolt:
                    max_jolt = jolt
        total += max_jolt
    return total


content = Path("d03.txt").read_text()
max_joltage = sum_max_joltage(content)
print(max_joltage)

17193


In [7]:
# part 2
def sum_max_joltage(content: str, k: int = 12) -> int:
    total = 0
    for line in content.strip().splitlines():
        digits = list(map(int, line.strip()))
        drop = len(digits) - k
        stack = []
        for d in digits:
            while stack and drop > 0 and stack[-1] < d:
                stack.pop()
                drop -= 1
            stack.append(d)
        # take only 1st k digits
        largest_digits = stack[:k]
        joltage = int(''.join(map(str, largest_digits)))
        total += joltage
    return total


content = Path("d03.txt").read_text()
max_joltage = sum_max_joltage(content)
print(max_joltage)

171297349921310


### Day 04: Printing Department

In [8]:
# part 1
from pathlib import Path
from itertools import product

def count_accessible_rolles(grid: list[str]) -> int:
    rows = range(len(grid))
    cols = range(len(grid[0]))
    positions = product(rows, cols)
    
    accessible_count = 0

    for r, c in positions:
        if grid[r][c] == "@":
            adjacent_rolls = 0

            
            for dr, dc in product([-1, 0, 1], repeat=2):
                if dr == 0 and dc == 0:
                    continue  # skip center cell
                nr, nc = r + dr, c + dc
                if nr in rows and nc in cols:
                    if grid[nr][nc] == "@":
                        adjacent_rolls += 1


            if adjacent_rolls < 4:
                accessible_count += 1
    
    return accessible_count

grid = Path("d04.txt").read_text().splitlines()

accessible_rolles = count_accessible_rolles(grid)
print(accessible_rolles)

1551


In [9]:
# part 2
def count_removed_rolls(grid: list[list[str]]) -> int:
    rows = range(len(grid))
    cols = range(len(grid[0]))
    positions = list(product(rows, cols))
    total_removed = 0

    while True:
        removed_this_round = 0

        for r, c in positions:
            if grid[r][c] == "@":
                adjacent_rolls = 0

                for dr, dc in product([-1, 0, 1], repeat=2):
                    if dr == 0 and dc == 0:
                        continue

                    nr, nc = r + dr, c + dc
                    if nr in rows and nc in cols:
                        if grid[nr][nc] == "@":
                            adjacent_rolls += 1

                if adjacent_rolls < 4:
                    grid[r][c] = "x"

        for r, c in positions:
            if grid[r][c] == "x":
                grid[r][c] = "."
                removed_this_round += 1

        total_removed += removed_this_round

        if removed_this_round == 0:
            break

    return total_removed


grid = Path("d04.txt").read_text().splitlines()
grid = [list(row) for row in grid]

removed_rolles = count_removed_rolls(grid)
print(removed_rolles)

9784


### Day 05: Cafeteria

Considerations Pt1
* ranges
    * are inclusive
    * can overlap / be adjacent -> if so, merge together
    * don't "roll out" because inefficient -> keep (begin, end) tuple
* check ID in ranges
    * check: begin < ID <= end (from tuples)
    * ~~loop over all merged ranges to find belonging range~~ brute-force 
    * find insert point on ranges list (binary search) -> more efficient


In [10]:
# part 1
from pathlib import Path
from bisect import bisect_right


def parse_input(lines: list[str]):
    ranges = [line for line in lines if "-" in line]
    ids = [line for line in lines if "-" not in line and line.strip()]
    ranges = [(int(b), int(e)) for b, e in (r.split("-") for r in ranges)]
    ids = [int(x) for x in ids]
    return ranges, ids


def merge_ranges(ranges: list[tuple[int, int]]) -> list[tuple[int, int]]:
    ranges.sort(key=lambda t: t[0])
    merged = [ranges[0]]
    for b, e in ranges[1:]:
        mb, me = merged[-1]
        if b <= me + 1:
            merged[-1] = (mb, max(me, e))
        else:
            merged.append((b, e))
    return merged


def is_in_any_range(num: int, merged: list[tuple[int, int]]) -> bool:
    starts = [b for b, _ in merged]
    idx = bisect_right(starts, num) - 1

    if idx < 0: # edge case possible?
        return False

    b, e = merged[idx]
    return b <= num <= e


def count_fresh_ids(lines: list[str]) -> int:
    ranges, ids = parse_input(lines)
    merged = merge_ranges(ranges)
    return sum(1 for num in ids if is_in_any_range(num, merged))


lines = Path("d05.txt").read_text().splitlines()
n_fresh = count_fresh_ids(lines)
print(n_fresh)

563


In [11]:
# part 2
from itertools import starmap

def subtract_range(b: int, e: int) -> int:
    return (e + 1) - b

def count_total_fresh(lines: list[str]) -> int:
    ranges, _ = parse_input(lines)
    merged = merge_ranges(ranges)
    return sum(starmap(subtract_range, merged))
    # return sum((e + 1 - b) for b, e in merged)

total_fresh = count_total_fresh(lines)

print(f"{total_fresh:_}")
print(total_fresh)

338_693_411_431_456
338693411431456


### Day 06: Trash Compactor

In [12]:
from pathlib import Path
from math import prod


def parse_input(lines: list[str]) -> tuple[list[list[int]], list[str]]:
    lines = [l.strip().split() for l in lines]

    operations = lines[-1]  

    numbers = [list(map(int, l))  for l in lines[:-1]]
    # transpose number using zip()
    numbers = [list(col) for col in zip(*numbers)]

    return numbers, operations


OPS = {"+": sum, "*": prod}


def compute_homework(numbers: list[list[int]], operations: list[str]) -> int:
    total = 0
    for nums, op in zip(numbers, operations):
        total += OPS[op](nums)
    
    return total


lines = Path("d06.txt").read_text().splitlines()
numbers, operations = parse_input(lines)
total = compute_homework(numbers, operations)

print(total)

7098065460541


In [13]:
# part 2
def transpose_input(lines: list[str]) -> list[str]:
    n_cols = len(lines[0])
    n_rows = len(lines)

    transposed = []
    for c in range(n_cols):
        buffer = ""
        for r in range(n_rows): 
            buffer += lines[r][c]
        transposed.append(buffer)

    return list(reversed(transposed))

def compute_homework(transposed: list[str]) -> int:
    total = 0
    numbers = []
    for row in transposed:
        row = row.strip()

        if row == "":
            # re-initialize number buffer
            numbers = []
            
        elif row.endswith(tuple(OPS.keys())):
            op = row[-1]
            num = int(row[:-1])
            numbers.append(num)
            total += OPS[op](numbers)
        
        else:
            numbers.append(int(row))

    return total

lines = Path("d06.txt").read_text().splitlines()
transposed = transpose_input(lines)
total = compute_homework(transposed)

print(total)

13807151830618


### Day 07: Laboratories

In [14]:
from pathlib import Path
from itertools import product

content = Path("d07.txt").read_text()
grid = content.splitlines()

rows = len(grid)
cols = len(grid[0])
positions = set(product(range(rows), range(cols)))

DIRECTIONS = {
    "left": (0, -1),
    "right": (0, 1),
    "down": (1, 0),
}

start_pos = (0, grid[0].index("S"))

beam_stack = [(start_pos, DIRECTIONS["down"])]
visited = set()
split_count = 0

while beam_stack:
    cur_pos, cur_dir = beam_stack.pop()
    nex_pos = (cur_pos[0] + cur_dir[0], cur_pos[1] + cur_dir[1])

    if nex_pos not in positions:
        # out of bounds
        continue

    state = (nex_pos[0], nex_pos[1], cur_dir)
    if state in visited:
        # avoid re-visting
        continue
    visited.add(state)

    cell = grid[nex_pos[0]][nex_pos[1]]
    # print(nex_pos, cell)

    if cell == "." or cell == "S":
        beam_stack.append((nex_pos, DIRECTIONS["down"]))

    elif cell == "^":
        split_count += 1
        # splitter moving left and right
        beam_stack.append((nex_pos, DIRECTIONS["left"]))
        beam_stack.append((nex_pos, DIRECTIONS["right"]))


print(split_count)

1649


In [15]:
# part 1: optimized
grid = Path("d07.txt").read_text().splitlines()

rows = len(grid)
cols = len(grid[0])
positions = product(range(rows), range(cols))

beam_set = {grid[0].index('S')}
split_count = 0
for row, col in positions:
    if grid[row][col] == "^" and col in beam_set:
        # on splitter: remove old beam
        beam_set.remove(col)
        # create new beams left & right
        beam_set.add(col - 1)
        beam_set.add(col + 1)
        split_count += 1

print(split_count)

1649


In [16]:
# part 2
from collections import defaultdict

grif = Path("d07.txt").read_text().splitlines()

rows = len(grid)
cols = len(grid[0])
positions = product(range(rows), range(cols))
beam_map = defaultdict(int, {grid[0].index('S'): 1})
timelines = 1

for row, col in positions:
    if grid[row][col] == "^" and col in beam_map:
        # on splitter: create new beams left & right
        beam_map[col - 1] += beam_map[col]
        beam_map[col + 1] += beam_map[col]
        timelines += beam_map[col]
        beam_map[col] = 0

print(timelines)

16937871060075


### Day 08: Playground
Considerations Pt1
* data: junction boxes in 3D space: x, y, z coordinates
* compute "closest by straight line" distance: pairwise Euclidean distance
    * $d_{\text{Euclid}}(p, q) = \sqrt{\sum_{i=1}^N (p_i - q_i)^2}$ for $N$ dimensions
    * potential optimize by alternative distance
        * e.g. squared distance
* process 1000 shortest pairs
    * if un-connected, connect them
    * if in different circuits, merge their sets
    * if already in same circuit, nothing happens (skip)
    * build *Union-Find Data Structure for Disjoint Set*
        * https://www.geeksforgeeks.org/dsa/introduction-to-disjoint-set-data-structure-or-union-find-algorithm/
* finally, get sizes of all circuits & multiply that of the 3 largest

In [17]:
# part 1
import heapq
from math import prod
import numpy as np


def compute_k_distances_pairs(X: np.ndarray, k: int) -> list[tuple[int, int, int]]:
    """Return format (distance, box_i, box_j)"""
    n = X.shape[0]
    
    maxheap = []
    for i in range(n):
        diffs = X[i] - X[i+1:] 
        squared = np.sum(diffs ** 2, axis=1)
        for offset, dis in enumerate(squared, start=1):
            j = i + offset
            item = (-dis, i, j)
            if len(maxheap) < k:
                heapq.heappush(maxheap, item)
            else:
                if item[0] > maxheap[0][0]:
                    heapq.heapreplace(maxheap, item)

    connections = [(-neg_d2.item(), i, j) for (neg_d2, i, j) in maxheap]
    connections.sort(key=lambda c: (c[0], c[1], c[2]))
    return connections


class Connector:
    def __init__(self, n: int):
        self.parent = list(range(n))
        self.rank = [0] * n
        self.size = [1] * n

    def find(self, a: int) -> int:
        while self.parent[a] != a:
            self.parent[a] = self.parent[self.parent[a]]
            a = self.parent[a]
        return a
    
    def connect(self, a: int, b: int) -> bool:
        ra, rb = self.find(a), self.find(b)
        if ra == rb:
            return False
        if self.rank[ra] < self.rank[rb]:
            self.parent[ra] = rb
            self.size[rb] += self.size[ra]
        elif self.rank[ra] > self.rank[rb]:
            self.parent[rb] = ra
            self.size[ra] += self.size[rb]
        else:
            self.parent[rb] = ra
            self.rank[ra] += 1
            self.size[ra] += self.size[rb]
        return True


matrix = np.genfromtxt("d08.txt", delimiter=",", dtype=np.int64)
K = 1000
distances = compute_k_distances_pairs(matrix, K)

n = matrix.shape[0]
connector = Connector(n)
for _, i, j in distances:
    connector.connect(i, j)

counts = {}
for v in range(n):
    r = connector.find(v)
    counts[r] = counts.get(r, 0) + 1

sizes = sorted(counts.values(), reverse=True)
top_3 = prod(sizes[:3])
top_3

75582

Considerations Pt2
* compute distance for all pairs
* iteratively, connect all pairs 
* track last pair to compute x-coord product

In [18]:
# part 2
def compute_all_distances_pairs(X: np.ndarray) -> list[tuple[float, int, int]]:
    """Return format (distance, box_i, box_j)"""
    n = X.shape[0]
    connections = []
    for i in range(n):
        diffs = X[i] - X[i+1:] 
        squared = np.sum(diffs ** 2, axis=1)
        for offset, d2 in enumerate(squared, start=1):
            j = i + offset
            connections.append((float(d2), i, j))
    connections.sort(key=lambda e: (e[0], e[1], e[2]))
    return connections

matrix = np.genfromtxt("d08.txt", delimiter=",", dtype=np.int64)
distances = compute_all_distances_pairs(matrix)

connector = Connector(n)
successful = 0
last_i = last_j = None
for _, i, j in distances:
    if connector.connect(i, j):
        successful += 1
        last_i, last_j = i, j
        if successful == n - 1:
            break

extension = int((matrix[last_i][0]) * (matrix[last_j][0]))
extension

59039696

### Day 09: Movie Theater

In [19]:
# part 1
from pathlib import Path

def compute_max_area(coords: list[tuple[int, int]]) -> int:
    n = len(coords)

    max_area = 0
    for i in range(n):
        for j in range(i + 1, n):
            x1, y1 = coords[i]
            x2, y2 = coords[j]
            width  = abs(x2 - x1) + 1
            height = abs(y2 - y1) + 1
            area = width * height
            if area > max_area:
                max_area  = area
    
    return max_area


lines = Path("d09.txt").read_text().splitlines()
coords = [tuple(map(int, l.split(","))) for l in lines]
max_area = compute_max_area(coords)
print(max_area)

4737096935


In [20]:
# part 2: exploration
import numpy as np

coords = np.array(coords)
x_vals = coords[:, 0].copy()
y_vals = coords[:, 1].copy()

min_x, max_x = int(x_vals.min()), int(x_vals.max())
min_y, max_y = int(y_vals.min()), int(y_vals.max())

cols = max_x - min_x
rows = max_y - min_y
cells = cols * rows

print(f"n = {coords.shape[0]}")
print(f"{min_x = }, {max_x = } → {cols = }")
print(f"{min_y = }, {max_y = } → {rows = }")
print(f"cells = cols × rows ≈ {cells:.2e}")

n = 496
min_x = 1624, max_x = 98248 → cols = 96624
min_y = 1646, max_y = 98294 → rows = 96648
cells = cols × rows ≈ 9.34e+09


Considerations Pt2

Danger: Size Explosion
* get an overview of size
    ```
    n = 496
    min_x = 1624, max_x = 98248 → cols = 96624
    min_y = 1646, max_y = 98294 → rows = 96648
    cells = cols × rows ≈ 9.34 billion
    ```
* working with a full grid of 9.34 billion cells will never finish

In [21]:
# part 2
from pathlib import Path
from itertools import chain

def compute_max_area(coords: list[tuple[int, int]]) -> int:
    return max(
        (
            (abs(x1 - x2) + 1) * (abs(y1 - y2) + 1)
            for i, (x1, y1) in enumerate(coords)
            for x2, y2 in coords[:i]
            if all(
                max(x3, x4) <= min(x1, x2)
                or max(x1, x2) <= min(x3, x4)
                or max(y3, y4) <= min(y1, y2)
                or max(y1, y2) <= min(y3, y4)
                for (x3, y3), (x4, y4) in zip(coords, chain(coords[1:], coords[:1]))
            )
        ),
        default=0,
    )

lines = Path("d09.txt").read_text().splitlines()
coords = [tuple(map(int, l.split(","))) for l in lines]
max_area = compute_max_area(coords)
print(max_area)

1644094530


### Day 10: Factory

In [22]:
# part 1
from collections import deque
from pathlib import Path
import re
from collections import namedtuple

Machine = namedtuple("machine", ["lights", "buttons", "joltage"])

def parse_input(content: str) -> list[Machine]:
    pattern = re.compile(
        r"\[(?P<lights>[.#]+)\]\s+"           # indicator lights
        r"(?P<buttons>(?:\(\d+(?:,\d+)*\)\s*)+)"  # button groups
        r"\{(?P<joltage>[\d,]+)\}"           # joltage requirements
    )

    machines  = []
    for line in content.splitlines():
        match = pattern.match(line)
        if match:
            lights = match.group("lights")

            buttons_raw = match.group("buttons")
            buttons = re.findall(r"\(([\d,]+)\)", buttons_raw)
            buttons = [list(map(int, b.split(","))) for b in buttons]

            raw_joltage = match.group("joltage")
            joltage = list(map(int, raw_joltage.split(",")))

            machine = Machine(lights=lights, buttons=buttons, joltage=joltage)
            machines.append(machine)

    return machines


def find_min_button_presses(machines: list[Machine]) -> int:
    total_presses = 0
    for machine in machines:
        # convert target lights to tuple of 0/1
        target_state = tuple(1 if c == '#' else 0 for c in machine.lights)
        num_lights = len(target_state)

        # initial state: all lights off
        initial_state = (0,) * num_lights

        # BFS queue: (current_state, presses_so_far)
        queue = deque([(initial_state, 0)])
        visited_states = set()

        while queue:
            current_state, press_count = queue.popleft()

            if current_state in visited_states:
                continue
            visited_states.add(current_state)

            # check: target reached / early stop
            if current_state == target_state:
                total_presses += press_count
                break

            # generate next states by pressing each button
            for button in machine.buttons:
                # toggle lights with this button
                next_state = tuple(
                    current_state[i] if i not in button else 1 - current_state[i]
                    for i in range(num_lights)
                )
                queue.append((next_state, press_count + 1))

    return total_presses


content = Path("d10.txt").read_text()
machines = parse_input(content)
total_presses = find_min_button_presses(machines)
print(total_presses)

401


In [23]:
# part 2
import numpy as np
from scipy.optimize import linprog


def build_matrix(buttons: list[list[int]], num_counters: int) -> np.ndarray:
    # build matrix A: rows = counters, columns = buttons
    mat = np.zeros((num_counters, len(buttons)), dtype=np.int32)
    for col, button in enumerate(buttons):
        for row in button:
            mat[row, col] = 1
    return mat


def find_min_joltage_presses(joltage: list[int], buttons: list[list[int]]) -> int:
    # compute minimal presses using integer linear programming
    num_counters = len(joltage)
    mat_a = build_matrix(buttons, num_counters)
    mat_b = np.array(joltage)

    # objective: minimize sum of presses
    c = np.ones(len(buttons))

    # solve ILP: A_eq * x = b_eq, x >= 0, integer
    result = linprog(
        c=c,
        A_eq=mat_a,
        b_eq=mat_b,
        bounds=[(0, None)] * len(buttons),
        integrality=np.ones(len(buttons)),  # enforce integer solution
        method="highs"
    )

    if not result.success:
        raise ValueError("No solution found")

    return int(round(result.fun))


machines = parse_input(content)
total_presses = 0
for machine in machines:
    presses = find_min_joltage_presses(machine.joltage, machine.buttons)
    total_presses += presses

print(total_presses)

15017


### Day 11: Reactor

In [24]:
from pathlib import Path
from functools import cache

def parse_input(content: str) -> dict[str, list[str]]:
    devices = {}
    for line in content.splitlines():
        key, val = line.split(": ")
        val = val.split(" ")
        devices[key] = val
    return devices

  
def part1(devices: dict[str, list[str]]) -> int:

    @cache
    def count(key: str) -> int:
        if key == "out":
            return 1
        return sum(count(dev) for dev in devices[key])

    return count("you")

content = Path("d11.txt").read_text()
devices = parse_input(content)
part1(devices)

670

In [25]:
def part2(devices: dict[str, list[str]]) -> int:

    @cache
    def count(key: str, b_dac: bool = False, b_fft: bool = False) -> int:
        match key:
            case "out":
                return 1 if b_dac and b_fft else 0
            case "dac":
                b_dac = True
            case "fft":
                b_fft = True

        return sum(count(dev, b_dac, b_fft) for dev in devices[key])

    return count("svr")

part2(devices)

332052564714990

### Day 12: Christmas Tree Farm

In [26]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class Region:
    width: int
    height: int
    quantities: list[int]

    def area(self) -> int:
        return self.width * self.height

@dataclass
class Present:
    ind: int
    shape: list[str]
    
    def cells(self):
        return sum(1 for row in self.shape for cell in row if cell == "#")


def parse_input(content: str) -> tuple[list[Region], list[Present]]:
    presents = content.split("\n\n")
    regions = presents.pop()
    regions_list = []
    for r in regions.split("\n"):
        shape, quantities = r.split(": ")
        shape = tuple(map(int, shape.split("x")))
        quantities = list(map(int, quantities.split(" ")))
        regions_list.append(Region(width=shape[0], height=shape[1], quantities=quantities))

    presents_list = []
    for p in presents:
        ind, shape = p.split(":\n")
        ind = int(ind)
        shape = shape.split("\n")
        presents_list.append(Present(ind=ind, shape=shape))

    return regions_list, presents_list


def fit_region(region: Region, presents_list: list[Present]) -> bool:
    all_presents = [q * p.cells() for q, p in zip(region.quantities, presents_list)]
    n_all_presents = sum(all_presents)
    return region.area() >= n_all_presents


content = Path("d12.txt").read_text()
regions_list, presents_list = parse_input(content)

how_many_fit = 0
for region in regions_list:
    if fit_region(region, presents_list):
        how_many_fit += 1
    
how_many_fit

567